In [18]:
import graphviz as gv
import matplotlib.pyplot as plt
from math import *
from collections import deque
import os
from pathlib import Path


def check_path():
  CWD = f'{os.getcwd()}'
  if Path(CWD).name == 'english-words-graph':
    os.chdir(r'./project/')
    return f'{os.getcwd()}'
  if Path(CWD).name == 'data_generation':
    os.chdir(r'../')
    return f'{os.getcwd()}'
  return CWD

CWD = check_path()
DATASET1 = CWD + r'\datasets\SimVerb-3500.txt'
DATASET2 = CWD + r'\datasets\wordsim_relatedness_goldstandard.txt'
DATASET3 = CWD + r'\datasets\men.txt'

def normalize(n, top, c):
  return c*(n)/top

def read_data():
  word_list = []
  relations = []
  with open(DATASET1) as file:
    lines = file.readlines()
    for line in lines:
      line = line.split('	')
      first_word = line[0]
      second_word = line[1]
      relatedness = line[3]
      word_list.append(first_word)
      word_list.append(second_word)
      relations.append([first_word,[second_word,relatedness]])

  with open(DATASET2) as file:
    lines = file.readlines()
    for line in lines:
      line = line.split('	')
      first_word = line[0]
      second_word = line[1]
      relatedness = line[2]
      word_list.append(first_word)
      word_list.append(second_word)
      w = [second_word,relatedness[:-2]]
      relations.append([first_word, w])

  with open(DATASET3) as file:
    lines = file.readlines()
    for line in lines:
      line = line.split(' ')
      first_word = line[0]
      second_word = line[1]
      relatedness = normalize(float(line[2]), 9.96, 1)
      word_list.append(first_word)
      word_list.append(second_word)
      w = [second_word,relatedness]
      relations.append([first_word, w])
  return list(set(word_list)), relations

def build_graph(G):
  words_dict = dict()
  for u in range(len(G)):
    word = G[u][0]
    related = G[u][1][0]
    if word in words_dict:
      existing = words_dict[word]
      existing.append(G[u][1])
      words_dict[word] = existing
    else:
      words_dict[word] = [G[u][1]]
    if related in words_dict:
      existing = words_dict[related]
      existing.append([G[u][0], G[u][1][1]])
      words_dict[related] = existing
    else:
      words_dict[related] = [[G[u][0], G[u][1][1]]]
  return words_dict


def BFS_findsubgraph(G, source = ''):
  newG = dict()
  explored = set()
  lengths = []
  if source == '':
    source = list(G.keys())[0]
  queue = deque([])
  queue.extend(G[source])

  while(len(queue) > 0):
    current = queue.popleft()
    newG[current[0]] = G[current[0]]
    for neighbor in G[current[0]]:
      if neighbor[0] not in explored:
        queue.append(neighbor)
        explored.add(neighbor[0])
  keys = list(newG.keys())
  print(f'Number of nodes: {len(newG)}. \nExample node and edge ([node] - [adjacent node, weight]) [{keys[100]}] - [{G[keys[100]][0][0]}, {G[keys[100]][0][1]}]')
  return newG

def convert(G):
  new_G = []
  keys = list(G.keys())
  for i in range(len(G)):
    for j in range(len(G[keys[i]])):
      new_G.append([keys[i], G[keys[i]][j]])
  return new_G


In [19]:
word_list, relations = read_data()
relations = BFS_findsubgraph(build_graph(relations))

max = 0
for k in relations.keys():
  for i in range(len(relations[k])):
    if float(relations[k][i][1]) > max:
        max = float(relations[k][1][1])

Number of nodes: 1687. 
Example node and edge ([node] - [adjacent node, weight]) [pick] - [elect, 7.47]


In [20]:
'''
def draw_lines(adj: dict):
    prevx = 0
    prevy = 0
    points_1 = []
    points_2 = []
    x=0
    y=0
    #relu
    for k in adj.keys():     
      grade = len(adj[k])
      angle_div = pi/grade
      angle = pi/4
      for n in range(grade):
        angle += n*angle_div
        radiusx = float(adj[k][n][1]) if float(adj[k][n][1]) > 0 else 0.1
        radiusy = float(adj[k][n][1]) if float(adj[k][n][1]) > 0 else 0.1
        x = prevx + cos(angle)/normalize(radiusx, max, 20)
        y = prevy + sin(angle)/normalize(radiusy, max, 20)
        points_1.append([prevx,prevy])
        points_2.append([x,y])
      prevx = x
      prevy = y
    return points_1, points_2
'''
def calculate_position(adj: dict):
    prevx = 0
    prevy = 0
    points_x = []
    points_y = []
    annotations = []
    x=0
    y=0
    #relu
    counter = 0
    for k in adj.keys():  
      grade = 0   
      for n in range(len(adj[k])):
        if adj[k][n][0] not in annotations:
          grade += 1
      if grade == 0:
        continue
      angle_div = pi/grade
      angle = pi/4
      #what if already grade = 0 bc tkaen
      for n in range(grade):
        if adj[k][n][0] in annotations:
          counter += 1
          continue
        angle += n*angle_div
        radiusx = float(adj[k][n][1]) if float(adj[k][n][1]) > 0 else 0.1
        radiusy = float(adj[k][n][1]) if float(adj[k][n][1]) > 0 else 0.1

        #is x really prevx

        x = prevx + cos(angle)/normalize(radiusx, max, 10)
        y = prevy + sin(angle)/normalize(radiusy, max, 10)
        points_x.append(x)
        points_y.append(y)
        annotations.append(adj[k][n][0])
      prevx = x
      prevy = y
    return points_x, points_y, annotations

X, Y, annotations = calculate_position(relations)

#points1, points2 = draw_lines(relations)

plt.figure(figsize=(100,100))
plt.scatter(X,Y,s=100,color="red")
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Scatter Plot with annotations",fontsize=15)
for i, label in enumerate(annotations):
    plt.annotate(label, (X[i], Y[i]))
    #x_values = [points1[i][0], points2[i][0]]
    #y_values = [points1[i][1], points2[i][1]]
    #plt.plot(x_values, y_values, 'bo', linestyle="-")
